In [5]:
import requests

response = requests.get('http://localhost:3000/api/v0.1.2/collecteds')
data = response.json()
response_2 = requests.get('http://localhost:3000/api/v0.1.2/detections')
data_2 = response_2.json()

print('collecteds')
print(data)
print('detections')
print(data_2)

collecteds
{'D1': {'x_axis_labels': ['00:00:01', '00:00:02', '00:00:03', '00:00:04', '00:00:05', '00:00:06', '00:00:07', '00:00:08', '00:00:09', '00:00:10'], 'y_axis_labels': [51, 52, 53, 54, 50, 90, 91, 92, 93, 94, 100]}}
detections
{'D1': {'max': {'condition': 'below', 'threshold': 100}, 'min': {'condition': 'above', 'threshold': 50}}, 'D2': {}, 'D3': {}, 'D4': {}}


In [1]:
import sys
import requests
from PySide6.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton, QTextEdit
from PySide6.QtCharts import QChart, QChartView, QLineSeries, QScatterSeries, QDateTimeAxis, QValueAxis
from PySide6.QtCore import Qt, QDateTime
from PySide6.QtGui import QPainter

class DataVisualizer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.detections = {}
        self.initUI()

    def initUI(self):
        self.main_widget = QWidget()
        self.setCentralWidget(self.main_widget)
        self.layout = QVBoxLayout(self.main_widget)

        # 按钮用于加载数据
        self.load_button = QPushButton('Load Data')
        self.load_button.clicked.connect(self.load_data)
        self.layout.addWidget(self.load_button)
        self.load_button.setFixedSize(100, 40)

        # 初始化图表
        self.chart = QChart()
        self.chart_view = QChartView(self.chart)
        self.chart_view.setRenderHint(QPainter.Antialiasing)
        self.layout.addWidget(self.chart_view)

        # 添加用于显示警示值的文本框
        self.alert_text_box = QTextEdit()
        self.alert_text_box.setReadOnly(True)
        self.alert_text_box.setFixedHeight(150)
        self.layout.addWidget(self.alert_text_box)

        self.resize(1280, 768)
        self.setWindowTitle('Data Visualization')

    def load_data(self):
        # 调用 API 获取 detections 数据
        detections_response = requests.get('http://localhost:3000/api/v0.1.2/detections')
        self.detections = detections_response.json()

        # 调用 API 获取 collecteds 数据
        collecteds_response = requests.get('http://localhost:3000/api/v0.1.2/collecteds')
        collecteds_data = collecteds_response.json()

        # 使用获取的数据创建折线图
        self.update_chart(collecteds_data)

    def update_chart(self, data):
        self.chart = QChart()  # 重新创建图表以清除旧数据
        self.chart_view.setChart(self.chart)

        axisX = QDateTimeAxis()
        axisX.setFormat("HH:mm:ss")
        axisX.setTitleText("Time")
        self.chart.addAxis(axisX, Qt.AlignBottom)

        axisY = QValueAxis()
        axisY.setTitleText("Value")
        self.chart.addAxis(axisY, Qt.AlignLeft)

        alert_values = []

        for register, values in data.items():
            series = QLineSeries()
            series.setName("Data")
            scatter_series = QScatterSeries()
            scatter_series.setName("Detection Points")
            scatter_series.setMarkerSize(10.0)

            if register in self.detections:
                detection = self.detections[register]
                if 'max' in detection:
                    self.add_alert_line(axisX, detection['max']['threshold'])
                if 'min' in detection:
                    self.add_alert_line(axisX, detection['min']['threshold'])

            for i, y in enumerate(values['y_axis_labels']):
                if i < len(values['x_axis_labels']):
                    time_str = values['x_axis_labels'][i]
                    time = QDateTime.fromString(f"2023-01-01T{time_str}", "yyyy-MM-ddTHH:mm:ss")
                    series.append(time.toMSecsSinceEpoch(), y)

                    if register in self.detections:
                        detection = self.detections[register]
                        if (('max' in detection and detection['max']['condition'] == 'below' and y > detection['max']['threshold']) or
                            ('min' in detection and detection['min']['condition'] == 'above' and y < detection['min']['threshold'])):
                            scatter_series.append(time.toMSecsSinceEpoch(), y)
                            alert_values.append((time_str, y))  # 记录超出阈值的数据点

            self.chart.addSeries(series)
            series.attachAxis(axisX)
            series.attachAxis(axisY)
            self.chart.addSeries(scatter_series)
            scatter_series.attachAxis(axisX)
            scatter_series.attachAxis(axisY)

        # 展示 alert_values
        alert_text = "Alert Values:\n"
        for time_str, value in alert_values:
            alert_text += f"Time: {time_str}, Value: {value}\n"
        self.alert_text_box.setText(alert_text)

        self.chart.legend().show()
        self.chart.setTitle(f'Data Visualization - {register}')
    
    def add_alert_line(self, y_value):
        alert_line = QLineSeries()
        alert_line.append(0, y_value)  # 假设 X 轴的起始值为 0
        alert_line.append(self.chart.plotArea().width(), y_value)  # 使用图表区域的宽度作为 X 轴的结束值
        self.chart.addSeries(alert_line)
        alert_line.attachAxis(self.chart.axisX())
        alert_line.attachAxis(self.chart.axisY())
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DataVisualizer()
    ex.show()
    sys.exit(app.exec())


C:\Users\User\AppData\Local\Temp\ipykernel_2492\3303686472.py:118: DeprecationWarning: Function: 'QChart.axisY(QAbstractSeries * series) const' is marked as deprecated, please check the documentation for more information.
  alert_line.attachAxis(self.chart.axisY())


SystemExit: 0

c:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
